# Volco Simulation in Jupyter Notebook

This notebook demonstrates how to run the Volco simulation directly from a Jupyter notebook using the new `run_simulation` function.

In [ ]:
import json
import sys
import os
import numpy as np

# Add the parent directory to the path so we can import volco
# This only applies if this notebook is run directly from the 
# examples directory. If you run it from the root directory, you
# don't need this.
sys.path.append(os.path.abspath('..'))

from volco import run_simulation

## Example 1: Using File Paths

As with running volco from the command line or a python file, you can use file paths for settings and gcode if you prefer:

In [ ]:
# Define file paths
gcode_path = '../examples/gcode_example.gcode'
printer_config_path = '../examples/printer_settings.json'
sim_config_path = '../examples/simulation_settings.json'

# Run the simulation using file paths
voxel_space, output = run_simulation(
    gcode_path=gcode_path,
    printer_config_path=printer_config_path,
    sim_config_path=sim_config_path
)

# the simulation output can be used to generate a mesh, export 
# to STL, or visualize the voxel space (shown later in this doc)

# Generate the mesh using the mesh module functionality
mesh = output.generate_mesh()

# Export to STL if needed
output.export_mesh_to_stl(mesh)

# Visualize the voxel space
fig = output.visualize_mesh(mesh, visualizer='trimesh', color_scheme='cyan_blue')
fig.show()

## Example 2: Using Python Variables

You can also pass the G-code content and configuration dictionaries directly:

In [ ]:
# Load G-code content from file
with open('../examples/gcode_example.gcode', 'r') as f:
    gcode_content = f.read()

# add new lines to the G-code 
gcode_content += '\nG0 F7200 X12.0 Y10.0 Z1.0'
gcode_content += '\nG1 F1000 X14.0 E0.133041'

# Load printer configuration from file
with open('../examples/printer_settings.json', 'r') as f:
    printer_config = json.load(f)

# Load simulation configuration from file
with open('../examples/simulation_settings.json', 'r') as f:
    sim_config = json.load(f)

# Run the simulation using Python variables
voxel_space, output = run_simulation(
    gcode=gcode_content,
    printer_config=printer_config,
    sim_config=sim_config
)
mesh = output.generate_mesh()
output.export_mesh_to_stl(mesh)

## Example 3: Creating Configuration Dictionaries Directly

You can also create the configuration dictionaries directly in Python:

In [ ]:
# Create printer configuration dictionary
custom_printer_config = {
    "nozzle_jerk_speed": 1.0,
    "extruder_jerk_speed": 5.0,
    "nozzle_acceleration": 120.0,
    "extruder_acceleration": 1200.0,
    "feedstock_filament_diameter": 1.75,
    "nozzle_diameter": 0.4
}

# Create simulation configuration dictionary
custom_sim_config = {
    "voxel_size": 0.05,
    "step_size": 0.1,
    "x_offset": 2.0,
    "y_offset": 2.0,
    "z_offset": 0.5,
    "simulation_name": "Test_volco",
    "results_folder": "Results_volco",
    "radius_increment": 0.1,
    "solver_tolerance": 0.0001,
    "x_crop": ["all", "all"],
    "y_crop": ["all", "all"],
    "z_crop": [0.0, "all"],
    "consider_acceleration": True
}

gcode_content = """M83
G0 X10 Y10 Z0.3
G1 F7200 X10.0 Y10.0 Z0.3
G1 F1000 X14.0 E0.133041
G0 F7200 X12.0 Y8.0 Z0.5
G1 F1000 Y12.0 E0.133041
G1 F7200 X10.0 Y10.0 Z0.7
G1 F1000 X12.0 E0.067
G1 F1000 X14 Y8.0 E0.1
G1 Z4 E0.2"""

# Run the simulation with custom configurations
voxel_space, output = run_simulation(
    gcode=gcode_content,
    printer_config=custom_printer_config,
    sim_config=custom_sim_config,
)
mesh = output.generate_mesh()
output.export_mesh_to_stl(mesh)

## Working with the Results

The `run_simulation` function returns the `voxel_space` and `output` objects (SimulationOutput instance), which you can use for further analysis:

In [ ]:
# Access voxel space dimensions
print(f"Voxel space dimensions: {voxel_space.dimensions}")

In [ ]:
# Visualize the mesh with plotly or trimesh amnd color_scheme 'cyan_blue' or 'viridis'
fig = output.visualize_mesh(mesh, visualizer='plotly', color_scheme='cyan_blue')
fig.show()